In [3]:
import numpy as np
import tensorflow as tf
from tf_agents.networks.q_network import QNetwork
import tf_agents.networks.network as network
from tf_agents.specs import tensor_spec
from snake_game import SnakeGame
from scene import Scene
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input

scene = Scene(init_randomly=True)
episodes_count = 50000
random_episodes = 20000


In [4]:
from tf_agents.environments.tf_py_environment import TFPyEnvironment

env = SnakeGame(scene)
env = TFPyEnvironment(env)


In [5]:
class MyQNetwork(network.Network):
    def __init__(self,
                 input_tensor_spec,
                 action_spec,
                 name='MyQNetwork'):
        super(MyQNetwork, self).__init__(
            input_tensor_spec=input_tensor_spec,
            state_spec=(),
            name=name)
        
        self._action_spec = action_spec

        input_shape = input_tensor_spec.shape
        
        self._model = Sequential([
            Input(shape=input_shape),
            Conv2D(32, (2, 2), 1, activation='relu', kernel_initializer='he_normal'),
            Conv2D(64, (2, 2), 1, activation='relu', kernel_initializer='he_normal'),
            Flatten(),
            Dense(128, activation='relu', kernel_initializer='he_normal'),
            Dense(16, activation='relu', kernel_initializer='he_normal'),
            Dense(4, activation='linear')
        ])

    def call(self, observations, step_type=None, network_state=(), training=False):
        output = self._model(observations)
        return output, network_state

q_net = MyQNetwork(
    env.observation_spec(),
    env.action_spec())


In [6]:
# Create the agent
from tf_agents.agents.dqn.dqn_agent import DdqnAgent
from tensorflow import keras
from keras.optimizers import Adam
from keras.losses import Huber
from tf_agents.trajectories import TimeStep

train_step_counter = tf.Variable(0)
optimizer = Adam(learning_rate=0.003)

# epsilon = lambda train_step: (1 / (tf.cast(tf.linspace(1, 8, episodes_count)**3, tf.float32)))[train_step]
epsilon = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1.0,
    decay_steps=episodes_count,
    end_learning_rate=0.01
)

agent = DdqnAgent(
    time_step_spec=env.time_step_spec(),
    action_spec=env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=Huber(reduction="none"),
    gamma=tf.constant(0.95, dtype=tf.float32),
    train_step_counter=train_step_counter,
    epsilon_greedy=lambda: epsilon(train_step_counter)
)


In [7]:
# Create the replay buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=env.batch_size,
    max_length=100000
)


In [8]:
# Create the training metrics
from tf_agents.metrics import tf_metrics

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric()
]


In [9]:
# Create the driver
from tf_agents.drivers import dynamic_step_driver

collect_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=agent.collect_policy,
    observers=[replay_buffer.add_batch] + train_metrics,
    num_steps=4
)

# Run a random policy to fill the replay buffer
from tf_agents.policies.random_tf_policy import RandomTFPolicy

initial_collect_policy = RandomTFPolicy(env.time_step_spec(), env.action_spec())
init_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=initial_collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=random_episodes
)

final_time_step, final_policy_state = init_driver.run()


In [10]:
# Create the dataset
dataset = replay_buffer.as_dataset(
    sample_batch_size=64,
    num_steps=2, # Capaz cambiar esto
    num_parallel_calls=3
).prefetch(3)


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [11]:
# Create the training loop

from tf_agents.utils.common import function
from tf_agents.trajectories import trajectory
from tf_agents.eval.metric_utils import log_metrics
import logging

logging.getLogger().setLevel(logging.INFO)

collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

def train_agent(n_iterations):
    time_step = None
    policy_state = agent.collect_policy.get_initial_state(env.batch_size)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        if iteration % 1000 == 0:
            print("Iteration: ", iteration)
            log_metrics(train_metrics)


In [12]:
train_agent(episodes_count)


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
INFO:absl: 
		 NumberOfEpisodes = 3
		 EnvironmentSteps = 4
		 AverageReturn = -31.66666603088379
		 AverageEpisodeLength = 1.0


Iteration:  0


INFO:absl: 
		 NumberOfEpisodes = 2752
		 EnvironmentSteps = 4004
		 AverageReturn = -40109.1015625
		 AverageEpisodeLength = 1.0


Iteration:  1000


...

Iteration:  47000


INFO:absl: 
		 NumberOfEpisodes = 60249
		 EnvironmentSteps = 192004
		 AverageReturn = -360503.40625
		 AverageEpisodeLength = 11.5


Iteration:  48000


INFO:absl: 
		 NumberOfEpisodes = 60481
		 EnvironmentSteps = 196004
		 AverageReturn = -338690.1875
		 AverageEpisodeLength = 15.899999618530273


Iteration:  49000


In [13]:
# Evaluate the agent

time_step = env.reset()
rewards = []
steps = 0

while not time_step.is_last() and steps < 200:
    steps += 1
    print("Map:\n", np.argmax(time_step.observation.numpy(), axis=3))
    action_step = agent.policy.action(time_step)
    print("Action: ", action_step.action.numpy())
    time_step = env.step(action_step.action)
    rewards.append(time_step.reward.numpy()[0])
    
print("Total reward: ", sum(rewards))
print("Total steps: ", steps)


Map:
 [[[0 0 0 0 0]
  [0 0 0 3 0]
  [0 0 0 0 0]
  [1 2 0 0 0]
  [0 0 0 0 0]]]
Action:  [1]
Map:
 [[[0 0 0 0 0]
  [0 0 0 3 0]
  [1 0 0 0 0]
  [2 0 0 0 0]
  [0 0 0 0 0]]]
Action:  [2]
Map:
 [[[0 0 0 0 0]
  [0 0 0 3 0]
  [2 1 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Action:  [1]
Map:
 [[[0 0 0 0 0]
  [0 1 0 3 0]
  [0 2 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Action:  [2]
Map:
 [[[0 0 0 0 0]
  [0 2 1 3 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Action:  [2]
Map:
 [[[0 0 0 0 0]
  [0 2 2 1 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 3 0 0]]]
Action:  [3]
Map:
 [[[0 0 0 0 0]
  [0 0 2 2 0]
  [0 0 0 1 0]
  [0 0 0 0 0]
  [0 0 3 0 0]]]
Action:  [3]
Map:
 [[[0 0 0 0 0]
  [0 0 0 2 0]
  [0 0 0 2 0]
  [0 0 0 1 0]
  [0 0 3 0 0]]]
Action:  [3]
Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 2 0]
  [0 0 0 2 0]
  [0 0 3 1 0]]]
Action:  [0]
Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 2 0]
  [3 0 0 2 0]
  [0 0 1 2 0]]]
Action:  [0]
Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [3 0 0 2 0]
  [0 1 2 2 0]]]
Action:  [0]

In [14]:
# Count the average steps per episode

n = 200
total_steps = 0
total_reward = 0
steps = 0
total_500 = 0

for i in range(n):
    if i % 10 == 0:
        print("Episode: ", i)
        
    time_step = env.reset()
    steps = 0
    while not time_step.is_last() and steps < 500:
        steps += 1
        action_step = agent.policy.action(time_step)
        time_step = env.step(action_step.action)
        reward = time_step.reward.numpy()[0]
        total_reward += reward
    
    if steps == 500:
        total_500 += 1
    else:
        total_steps += steps
        
if steps == 500:
    print("500 episodes reached")
    
print("Average steps per episode: ", total_steps / (n - total_500))
print("Average reward per episode: ", total_reward / n)
print("Total 500 episodes: ", total_500)


Episode:  0
Episode:  10
Episode:  20
Episode:  30
Episode:  40
Episode:  50
Episode:  60
Episode:  70
Episode:  80
Episode:  90
Episode:  100
Episode:  110
Episode:  120
Episode:  130
Episode:  140
Episode:  150
Episode:  160
Episode:  170
Episode:  180
Episode:  190
Average steps per episode:  23.31
Average reward per episode:  130.65
Total 500 episodes:  0


In [15]:
# Save the agent
import os
from tf_agents.policies.policy_saver import PolicySaver

saver = PolicySaver(agent.policy, batch_size=None)
saver.save("policy")


TypeError: this __dict__ descriptor does not support '_DictWrapper' objects